## Loading dataset
We will get our dataset from kaggle, for this you will have to add kaggle.json file in the `/content` directory. You can find the json [here](https://www.kaggle.com/settings/account).

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

## Un-zipping the dataset
Since the dataset downloaded from kaggle is in zip file, we will have to un-zip it.

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, regularizers
from keras.layers import (
    Dense,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dropout
)

In [ ]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

val_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

In [ ]:
def normalization(image, label):
  image = tf.cast(image / 255. , tf.float32)
  return image, label

In [ ]:
train_ds = train_ds.map(normalization)
val_ds = val_ds.map(normalization)

In [ ]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(256,256,3)),
    MaxPooling2D(pool_size=(3,2), strides=2, padding='valid'),

    Conv2D(
        64,
        kernel_size=(3,3),
        padding='valid',
        activation='relu',
    ),
    MaxPooling2D(pool_size=(3,2), strides=2, padding='valid'),

    Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu'),
    MaxPooling2D(pool_size=(3,2), strides=2, padding='valid'),

    Conv2D(
        256,
        kernel_size=(3,3),
        padding='valid',
        activation='relu',
      ),
    MaxPooling2D(pool_size=(3,2), strides=2, padding='valid'),

    Flatten(),

    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='sigmoid'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
)

In [ ]:
# checking the model 
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
!wget https://i.natgeofe.com/n/548467d8-c5f1-4551-9f58-6817a8d2c45e/NationalGeographic_2572187_square.jpg

In [ ]:
import cv2

In [ ]:
cat_image_test = cv2.imread('/content/NationalGeographic_2572187_square.jpg')

In [ ]:
plt.imshow(cat_image_test) # for showing cute lil cat

In [ ]:
cat_image_test = cv2.resize(cat_image_test,(256,256))
cat_image_test = cat_image_test.reshape((1,256,256,3))

In [ ]:
model.predict(cat_image_test)